In [40]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

# Load dataset
file_path = "USDA.csv"
df = pd.read_csv(file_path)

# Drop any missing values in relevant columns
df.dropna(inplace=True)

clean_path = "cleaned_USDA_data.xlsx"
df.to_excel(clean_path, index=False)
print(f"Cleaned data saved to {clean_path}")

Cleaned data saved to cleaned_USDA_data.xlsx


In [41]:
df

,ID,Description,Calories,Protein,TotalFat,Carbohydrate,Sodium,SaturatedFat,Cholesterol,Sugar,Calcium,Iron,Potassium,VitaminC,VitaminE,VitaminD
0,1001,"BUTTER,WITH SALT",717.0,0.85,81.11,0.06,714.0,51.368,215.0,0.06,24.0,0.02,24.0,0.0,2.32,1.5
1,1002,"BUTTER,WHIPPED,WITH SALT",717.0,0.85,81.11,0.06,827.0,50.489,219.0,0.06,24.0,0.16,26.0,0.0,2.32,1.5
2,1003,"BUTTER OIL,ANHYDROUS",876.0,0.28,99.48,0.00,2.0,61.924,256.0,0.00,4.0,0.00,5.0,0.0,2.80,1.8
3,1004,"CHEESE,BLUE",353.0,21.40,28.74,2.34,1395.0,18.669,75.0,0.50,528.0,0.31,256.0,0.0,0.25,0.5
4,1005,"CHEESE,BRICK",371.0,23.24,29.68,2.79,560.0,18.764,94.0,0.51,674.0,0.43,136.0,0.0,0.26,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7053,80200,"FROG LEGS,RAW",73.0,16.40,0.30,0.00,58.0,0.076,50.0,0.00,18.0,1.50,285.0,0.0,1.00,0.2
7054,83110,"MACKEREL,SALTED",305.0,18.50,25.10,0.00,4450.0,7.148,95.0,0.00,66.0,1.40,520.0,0.0,2.38,25.2
7055,90240,"SCALLOP,(BAY&SEA),CKD,STMD",111.0,20.54,0.84,5.41,667.0,0.218,41.0,0.00,10.0,0.58,314.0,0.0,0.00,0.0
7056,90560,"SNAIL,RAW",90.0,16.10,1.40,2.00,70.0,0.361,50.0,0.00,10.0,3.50,382.0,0.0,5.00,0.0


# LPP

In [43]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

# --- Configuration ---
MIN_FOOD_ITEMS = 3
MAX_GRAMS_PER_ITEM = 500 # Maximum grams allowed for any single food item
FILE_PATH = "USDA.csv"
# Nutritional constraints (daily recommendations) - ADJUST AS NEEDED
CALORIES_MIN, CALORIES_MAX = 2000, 2500
PROTEIN_MIN = 50  # Minimum grams of protein
FAT_MAX = 70      # Maximum grams of fat
CARBS_MAX = 300   # Maximum grams of carbohydrates
# Add more constraints if desired (e.g., fiber_min = 25) and ensure column exists
NUTRIENT_COLS = ['Calories', 'Protein', 'TotalFat', 'Carbohydrate'] # Add 'Fiber' etc. if needed

# --- Load and Prepare Data ---
try:
    df = pd.read_csv(FILE_PATH)
except FileNotFoundError:
    print(f"Error: File not found at {FILE_PATH}")
    exit()

# Drop rows with missing values in essential nutrient columns
df.dropna(subset=NUTRIENT_COLS, inplace=True)

# Ensure nutrient columns are numeric, coercing errors
for col in NUTRIENT_COLS:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows again if coercion created new NaNs
df.dropna(subset=NUTRIENT_COLS, inplace=True)

# Reset index after dropping rows to ensure continuity
df.reset_index(drop=True, inplace=True)

if len(df) == 0:
    print("Error: No valid food data remaining after cleaning.")
    exit()

# --- Extract Data for LP ---
calories = df['Calories'].values
protein = df['Protein'].values
fat = df['TotalFat'].values
carbs = df['Carbohydrate'].values
# Example: fiber = df['Fiber'].values # If you added a fiber constraint

n_items = len(df)

# --- Define LP Problem ---
# Objective function: Find *any* feasible solution (coefficients are zero)
c = np.zeros(n_items)

# Constraints matrix (A_ub @ x <= b_ub)
# x represents units of 100g
A = [
    -calories,  # -calories * x <= -CALORIES_MIN  => calories * x >= CALORIES_MIN
    calories,   #  calories * x <=  CALORIES_MAX
    -protein,   # -protein * x <= -PROTEIN_MIN    => protein * x >= PROTEIN_MIN
    fat,        #  fat * x <=  FAT_MAX
    carbs,      #  carbs * x <=  CARBS_MAX
    # Example: -fiber # -fiber * x <= -fiber_min => fiber * x >= fiber_min
]
b = [
    -CALORIES_MIN,
    CALORIES_MAX,
    -PROTEIN_MIN,
    FAT_MAX,
    CARBS_MAX,
    # Example: -fiber_min
]

# **** MODIFIED BOUNDS ****
# Bounds: Amount of each food item must be non-negative (x_i >= 0)
# and not exceed MAX_GRAMS_PER_ITEM (x_i <= MAX_GRAMS_PER_ITEM / 100)
max_amount_100g_units = MAX_GRAMS_PER_ITEM / 100.0
initial_bounds = [(0, max_amount_100g_units) for _ in range(n_items)]
print(f"Setting individual food item limit: 0g to {MAX_GRAMS_PER_ITEM}g (0 to {max_amount_100g_units} units in solver)")


# --- Solve LP (Attempt 1) ---
print("\nAttempting to find an initial feasible solution with item limits...")
result = linprog(c, A_ub=A, b_ub=b, bounds=initial_bounds, method='highs')

# --- Heuristic to Encourage More Items ---
solution_found = False
if result.success:
    # Check how many items are used in the first solution
    epsilon = 1e-6 # Tolerance to consider an item "used"
    items_used_indices = np.where(result.x > epsilon)[0]
    num_items_used = len(items_used_indices)
    print(f"Initial solution found using {num_items_used} food item(s).")

    if num_items_used >= MIN_FOOD_ITEMS:
        print(f"Initial solution already meets the minimum requirement of {MIN_FOOD_ITEMS} items.")
        solution_found = True
    elif num_items_used > 0:
        print(f"Initial solution has only {num_items_used} item(s). Attempting to force more diversity...")
        # --- Modify Bounds and Re-solve (Attempt 2) ---
        # Start with the initial bounds (which already include the 0 to max_amount_100g_units limit)
        modified_bounds = initial_bounds.copy()

        # Limit the amount of the initially selected items further
        # Heuristic: cap them at half their current amount
        for idx in items_used_indices:
            current_amount = result.x[idx]
            # Calculate the new cap (half of current amount)
            # Ensure cap is slightly above zero if current amount is tiny but > epsilon
            cap = max(epsilon * 10, current_amount / 2.0)

            # The new cap will automatically be <= max_amount_100g_units
            # because current_amount was already <= max_amount_100g_units
            print(f"  - Further limiting '{df.loc[idx, 'Description']}' (ID: {df.loc[idx, 'ID']}) to {cap*100:.1f}g")
            modified_bounds[idx] = (0, cap) # Update the upper bound for this item

        print("\nRe-solving with modified bounds...")
        result = linprog(c, A_ub=A, b_ub=b, bounds=modified_bounds, method='highs')

        if result.success:
            items_used_indices_final = np.where(result.x > epsilon)[0]
            num_items_used_final = len(items_used_indices_final)
            print(f"Second attempt found a solution using {num_items_used_final} food item(s).")
            if num_items_used_final >= MIN_FOOD_ITEMS:
                solution_found = True
            else:
                 print(f"Warning: Could not force >= {MIN_FOOD_ITEMS} items while satisfying constraints using this heuristic.")
                 # We'll still show the result found, even if < MIN_FOOD_ITEMS
                 solution_found = True # Report the best we could find
        else:
            print("Re-solving failed. Could not find a feasible solution after limiting initial items.")
            result.success = False # Mark as failure for the final report
            solution_found = False

# --- Display Results ---
if solution_found and result.success:
    print("\nFeasible Diet Plan Found:")
    # result.x contains the amount of each food in 100g units
    df['Amount_g'] = result.x * 100  # Convert to grams

    # Select and display foods included in the diet
    feasible_diet = df[df['Amount_g'] > epsilon][['ID', 'Description', 'Amount_g']]
    num_items_final = len(feasible_diet)

    # Verify the individual limit wasn't exceeded (due to potential solver tolerances)
    if not feasible_diet.empty:
      max_found_amount = feasible_diet['Amount_g'].max()
      if max_found_amount > MAX_GRAMS_PER_ITEM + epsilon * 100: # Check with tolerance
          print(f"\nWARNING: Solver resulted in an item amount ({max_found_amount:.1f}g) slightly exceeding the limit of {MAX_GRAMS_PER_ITEM}g. This might be due to solver tolerances.")

    print(f"(Contains {num_items_final} food items)") # Report actual number in final plan
    print(feasible_diet.to_string(index=False))

    # Calculate and display the total nutrients for the found diet
    total_calories = np.dot(calories, result.x)
    total_protein = np.dot(protein, result.x)
    total_fat = np.dot(fat, result.x)
    total_carbs = np.dot(carbs, result.x)
    # Example: total_fiber = np.dot(fiber, result.x)

    print("\nTotal Nutrients in the Feasible Diet:")
    print(f"Calories: {total_calories:.2f} (Min: {CALORIES_MIN}, Max: {CALORIES_MAX})")
    print(f"Protein:  {total_protein:.2f} g (Min: {PROTEIN_MIN})")
    print(f"Fat:      {total_fat:.2f} g (Max: {FAT_MAX})")
    print(f"Carbs:    {total_carbs:.2f} g (Max: {CARBS_MAX})")
    # Example: print(f"Fiber:    {total_fiber:.2f} g (Min: fiber_min)")

elif result.status == 2: # Status 2 indicates infeasibility
     print("\nNo feasible solution found.")
     print(f"The nutritional constraints (including the {MAX_GRAMS_PER_ITEM}g per item limit) might be too strict, contradictory, or")
     print(f"it might be impossible to meet them using at least {MIN_FOOD_ITEMS} distinct foods from this dataset.")
     print("Consider relaxing the constraints (e.g., widening calorie range, increasing the per-item limit, lowering protein min, increasing fat/carb max) or reducing MIN_FOOD_ITEMS.")
else:
    # Handles other failure cases
    print(f"\nOptimization failed or did not find a suitable solution.")
    print(f"Solver status: {result.status} - {result.message}")

Setting individual food item limit: 0g to 500g (0 to 5.0 units in solver)

Attempting to find an initial feasible solution with item limits...
Initial solution found using 6 food item(s).
Initial solution already meets the minimum requirement of 3 items.

Feasible Diet Plan Found:
(Contains 6 food items)
   ID                                                  Description   Amount_g
 2051                                VANILLA EXTRACT,IMITN,ALCOHOL 500.000000
10874       PORK,CURED,HAM W/ NAT JUICES,SHANK,BONE-IN,LN,HTD,RSTD 192.694301
14010                  ALCOHOLIC BEV,DAIQUIRI,PREPARED-FROM-RECIPE 287.953368
14143 CARBONATED BEV,LO CAL,OTHER THAN COLA OR PEPPER,WO/ CAFFEINE 500.000000
14151 CARB BEV,LO CAL,OTHR THN COLA OR PEPPER,W/ ASPRT,CONTNS CAFF 500.000000
14201     COFFEE,BREWED FROM GROUNDS,PREP W/ TAP H2O,DECAFFEINATED 500.000000

Total Nutrients in the Feasible Diet:
Calories: 2000.00 (Min: 2000, Max: 2500)
Protein:  50.00 g (Min: 50)
Fat:      9.75 g (Max: 70)
Carbs:    32.

# Simulated annealing

In [44]:
import pandas as pd
import numpy as np
import random
import math # Needed for np.exp

# --- 1. Configuration & Nutritional Targets ---

# Load the dataset

# --- Define Your Dietary Goals (Approx. Average Indian Adult) ---
# Sources: ICMR guidelines vary, using representative values. Adjust as needed.
TARGETS = {
    'target_calories': 2200,   # Example target
    'calorie_tolerance': 250,  # Allowable deviation
    'min_protein_g': 75,     # RDA ~0.8-1.0 g/kg body weight
    'min_carbs_g': 275,      # ~50-60% of calories (assuming 2200 kcal)
    'max_fat_g': 73,         # ~25-30% of calories (max)
    'max_sodium_mg': 2000,   # Recommended limit often < 2300mg
    'min_calcium_mg': 1000,   # RDA
    'min_iron_mg': 17,       # RDA for adult male (higher for women)
}

# --- New Constraint Parameters ---
MIN_LARGE_ITEMS = 3
LARGE_ITEM_THRESHOLD_G = 500.0

# --- Simulated Annealing Parameters ---
INITIAL_TEMPERATURE = 1500.0 # Might need slightly higher temp for more constraints
COOLING_RATE = 0.997         # Slower cooling might help
NUM_ITERATIONS = 30000       # Increased iterations for complexity
NEIGHBOR_CHANGE_AMOUNT = 25  # Allow slightly larger changes
PENALTY_WEIGHT = 1000        # Base weight for nutrient penalties
LARGE_ITEM_PENALTY = PENALTY_WEIGHT * 50 # Significantly penalize failing the large item constraint

# --- 2. Helper Functions ---

def calculate_nutrition(diet, dataframe):
    """Calculates total nutrients for a given diet."""
    totals = {col: 0.0 for col in numeric_cols}
    totals['Description'] = []
    totals['TotalGrams'] = 0
    totals['LargeItemsInfo'] = [] # Store info about large items

    if not diet:
        return totals

    for food_id, grams in diet.items():
        if food_id in dataframe.index and grams > 0:
            try:
                food_data = dataframe.loc[food_id]
                scale_factor = grams / 100.0
                for col in numeric_cols:
                    totals[col] += food_data[col] * scale_factor
                description_str = f"{grams:.1f}g of {food_data['Description']}"
                totals['Description'].append(description_str)
                if grams >= LARGE_ITEM_THRESHOLD_G:
                     totals['LargeItemsInfo'].append(description_str)
                totals['TotalGrams'] += grams
            except KeyError:
                print(f"Warning: Food ID {food_id} not found in dataframe.")
            except Exception as e:
                 print(f"Warning: Error processing food ID {food_id}: {e}")

    return totals

def cost_function(diet, dataframe, targets, penalty_weight, large_item_penalty):
    """Calculates the cost (energy) of a diet. Lower is better."""
    if not diet:
        return float('inf') # Penalize empty diet heavily

    nutrition = calculate_nutrition(diet, dataframe)
    total_calories = nutrition['Calories']

    cost = 0
    penalty = 0

    # --- Apply Penalties for Nutrient Constraint Violations ---

    # Penalty for deviating from target calories
    calorie_diff = abs(total_calories - targets['target_calories'])
    if calorie_diff > targets['calorie_tolerance']:
         penalty += ((calorie_diff - targets['calorie_tolerance']) / 100) ** 2 # Scale down calorie penalty a bit

    # Penalty for missing minimum protein
    if nutrition['Protein'] < targets['min_protein_g']:
        penalty += (targets['min_protein_g'] - nutrition['Protein']) ** 2

    # Penalty for missing minimum carbohydrates
    if nutrition['Carbohydrate'] < targets['min_carbs_g']:
        penalty += ((targets['min_carbs_g'] - nutrition['Carbohydrate'])/10) ** 2 # Scale penalty

    # Penalty for exceeding maximum fat
    if nutrition['TotalFat'] > targets['max_fat_g']:
        penalty += (nutrition['TotalFat'] - targets['max_fat_g']) ** 2

    # Penalty for exceeding maximum sodium
    if nutrition['Sodium'] > targets['max_sodium_mg']:
        penalty += ((nutrition['Sodium'] - targets['max_sodium_mg']) / 100) ** 2 # Scale penalty

    # Penalty for missing minimum calcium
    if nutrition['Calcium'] < targets['min_calcium_mg']:
        penalty += ((targets['min_calcium_mg'] - nutrition['Calcium']) / 100) ** 2 # Scale penalty

    # Penalty for missing minimum iron
    if nutrition['Iron'] < targets['min_iron_mg']:
        penalty += (targets['min_iron_mg'] - nutrition['Iron']) ** 2 * 10 # Iron is small numbers, boost penalty


    # --- Apply Penalty for Structural Constraint (Large Items) ---
    large_items_count = sum(1 for grams in diet.values() if grams >= LARGE_ITEM_THRESHOLD_G)
    if large_items_count < MIN_LARGE_ITEMS:
        # Penalize based on how many items are missing
        penalty += large_item_penalty * (MIN_LARGE_ITEMS - large_items_count)


    # --- Combine Objective and Penalties ---
    # Objective: Minimize deviation from target calories (squared)
    # Cost = Objective + Weighted Penalties
    cost = (total_calories - targets['target_calories'])**2 + penalty * penalty_weight

    # Alternative Objective: Minimize total calories (if needed)
    # cost = total_calories + penalty * penalty_weight

    # Handle potential NaN/inf costs
    if math.isnan(cost) or math.isinf(cost):
        return float('inf')

    return cost


def generate_neighbor(current_diet, dataframe, change_amount):
    """Generates a slightly modified diet."""
    neighbor_diet = current_diet.copy()
    all_food_ids = list(dataframe.index)

    # Give a higher chance to modify existing items if the diet is large
    num_items = len(neighbor_diet)
    if num_items == 0:
        action = 'add_new'
    elif num_items < 5: # Encourage adding items early on
         action = random.choices(['add_new', 'remove_existing', 'increase', 'decrease'], weights=[0.5, 0.1, 0.2, 0.2], k=1)[0]
    else: # Focus more on adjustments later
        action = random.choices(['add_new', 'remove_existing', 'increase', 'decrease'], weights=[0.2, 0.2, 0.3, 0.3], k=1)[0]


    if action == 'add_new' and len(all_food_ids) > num_items:
        # Add a new food item with a random amount (could be small or large)
        attempts = 0
        while attempts < 10:
            food_id = random.choice(all_food_ids)
            if food_id not in neighbor_diet:
                # Introduce possibility of adding larger amounts directly
                amount = random.uniform(1, max(change_amount * 2, LARGE_ITEM_THRESHOLD_G / 4))
                neighbor_diet[food_id] = amount
                break
            attempts += 1

    elif action == 'remove_existing' and neighbor_diet:
        food_id_to_remove = random.choice(list(neighbor_diet.keys()))
        del neighbor_diet[food_id_to_remove]

    elif action == 'increase' and neighbor_diet:
        food_id_to_increase = random.choice(list(neighbor_diet.keys()))
        # Allow potentially large increases
        increase_by = random.uniform(1, change_amount * random.uniform(1, 3))
        neighbor_diet[food_id_to_increase] += increase_by

    elif action == 'decrease' and neighbor_diet:
        food_id_to_decrease = random.choice(list(neighbor_diet.keys()))
        decrease_by = random.uniform(1, change_amount)
        neighbor_diet[food_id_to_decrease] -= decrease_by
        if neighbor_diet[food_id_to_decrease] <= 0.1: # Use a small threshold
            del neighbor_diet[food_id_to_decrease]

    # Clean up tiny amounts that might remain after decrease
    neighbor_diet = {k: v for k, v in neighbor_diet.items() if v > 0.1}

    return neighbor_diet

# --- 3. Simulated Annealing Algorithm ---

def simulated_annealing(initial_diet, dataframe, targets,
                        initial_temp, cooling_rate, num_iterations,
                        change_amount, penalty_weight, large_item_penalty):
    """Performs the simulated annealing optimization."""

    current_diet = initial_diet
    current_cost = cost_function(current_diet, dataframe, targets, penalty_weight, large_item_penalty)
    best_diet = current_diet
    best_cost = current_cost
    temperature = initial_temp

    print(f"Initial Diet: {current_diet}")
    print(f"Initial Cost: {current_cost:.2f}")

    cost_history = [current_cost]

    for i in range(num_iterations):
        neighbor_diet = generate_neighbor(current_diet, dataframe, change_amount)
        neighbor_cost = cost_function(neighbor_diet, dataframe, targets, penalty_weight, large_item_penalty)

        delta_cost = neighbor_cost - current_cost

        # Acceptance Probability
        acceptance_prob = 1.0 if delta_cost < 0 else np.exp(-delta_cost / max(temperature, 1e-9)) # Avoid division by zero

        if random.random() < acceptance_prob:
            current_diet = neighbor_diet
            current_cost = neighbor_cost

            if current_cost < best_cost:
                best_diet = current_diet.copy() # Important: copy the dictionary
                best_cost = current_cost

        temperature *= cooling_rate
        cost_history.append(current_cost)

        if (i + 1) % (num_iterations // 20) == 0: # Print progress more often
             best_diet_large_items = sum(1 for g in best_diet.values() if g >= LARGE_ITEM_THRESHOLD_G)
             #print(f"Iter {i+1}/{num_iterations}, Temp: {temperature:.2f}, "
                  # f"Curr Cost: {current_cost:.2f}, Best Cost: {best_cost:.2f}, "
                   #f"Best Diet Large Items: {best_diet_large_items}/{MIN_LARGE_ITEMS}")

    print(f"\nFinished. Final Best Cost: {best_cost:.2f}")
    # Optional: Plot cost history (uncomment matplotlib import if needed)
    # import matplotlib.pyplot as plt
    # plt.plot(cost_history)
    # plt.xlabel("Iteration")
    # plt.ylabel("Cost")
    # plt.yscale('log') # Log scale often helpful for cost functions
    # plt.title("Cost Function Convergence")
    # plt.show()

    return best_diet, best_cost

# --- 4. Main Execution ---

if __name__ == "__main__":
    initial_diet = {}
    # Start with a slightly larger initial random diet to encourage exploration
    initial_food_ids = random.sample(list(df.index), k=min(8, len(df)))
    for food_id in initial_food_ids:
        initial_diet[food_id] = random.uniform(50, 250) # Start with varied amounts

    print("--- Starting Simulated Annealing ---")
    print(f"Targets (Approx. Indian Adult): {TARGETS}")
    print(f"Structural Constraint: >= {MIN_LARGE_ITEMS} items >= {LARGE_ITEM_THRESHOLD_G}g")
    print(f"Parameters: Temp={INITIAL_TEMPERATURE}, Cooling={COOLING_RATE}, Iterations={NUM_ITERATIONS}")

    best_diet_found, final_cost = simulated_annealing(
        initial_diet=initial_diet,
        dataframe=df,
        targets=TARGETS,
        initial_temp=INITIAL_TEMPERATURE,
        cooling_rate=COOLING_RATE,
        num_iterations=NUM_ITERATIONS,
        change_amount=NEIGHBOR_CHANGE_AMOUNT,
        penalty_weight=PENALTY_WEIGHT,
        large_item_penalty=LARGE_ITEM_PENALTY
    )

    print("\n--- Optimal Diet Found ---")
    if not best_diet_found:
        print("No feasible diet found.")
    else:
        final_nutrition = calculate_nutrition(best_diet_found, df)
        print(f"Final Cost (Lower is Better): {final_cost:.2f}")
        print(f"Total Grams: {final_nutrition['TotalGrams']:.1f}g")

        print(f"\nFoods ({len(best_diet_found)} items):")
        # Sort food items by grams, descending, for clarity
        sorted_diet_items = sorted(best_diet_found.items(), key=lambda item: item[1], reverse=True)
        large_item_count_final = 0
        for food_id, grams in sorted_diet_items:
             desc = df.loc[food_id, 'Description'] if food_id in df.index else f"ID {food_id}"
             marker = ""
             if grams >= LARGE_ITEM_THRESHOLD_G:
                 marker = f" (>= {LARGE_ITEM_THRESHOLD_G}g)"
                 large_item_count_final += 1
             print(f"- {grams:.1f}g of {desc}{marker}")


        print(f"\nStructural Constraint Check:")
        if large_item_count_final >= MIN_LARGE_ITEMS:
            print(f"  Met: Found {large_item_count_final} items >= {LARGE_ITEM_THRESHOLD_G}g.")
            # print("  Large items:")
            # for item_info in final_nutrition['LargeItemsInfo']:
            #     print(f"    - {item_info}")
        else:
             print(f"  NOT FULLY MET: Found only {large_item_count_final}/{MIN_LARGE_ITEMS} items >= {LARGE_ITEM_THRESHOLD_G}g.")
             print(f"   Consider increasing iterations, adjusting penalties, or relaxing constraints.")


        print("\nNutritional Summary (Values vs Targets):")
        print(f"- Calories:   {final_nutrition['Calories']:.1f} (Target: ~{TARGETS['target_calories']} +/- {TARGETS['calorie_tolerance']})")
        print(f"- Protein:    {final_nutrition['Protein']:.1f}g (Target: >= {TARGETS['min_protein_g']}g)")
        print(f"- Carbs:      {final_nutrition['Carbohydrate']:.1f}g (Target: >= {TARGETS['min_carbs_g']}g)")
        print(f"- Total Fat:  {final_nutrition['TotalFat']:.1f}g (Target: <= {TARGETS['max_fat_g']}g)")
        print(f"- Sodium:     {final_nutrition['Sodium']:.1f}mg (Target: <= {TARGETS['max_sodium_mg']}mg)")
        print(f"- Calcium:    {final_nutrition['Calcium']:.1f}mg (Target: >= {TARGETS['min_calcium_mg']}mg)")
        print(f"- Iron:       {final_nutrition['Iron']:.2f}mg (Target: >= {TARGETS['min_iron_mg']}mg)")
        # Add other nutrients if desired
        # print(f"- Potassium: {final_nutrition['Potassium']:.1f}mg")

--- Starting Simulated Annealing ---
Targets (Approx. Indian Adult): {'target_calories': 2200, 'calorie_tolerance': 250, 'min_protein_g': 75, 'min_carbs_g': 275, 'max_fat_g': 73, 'max_sodium_mg': 2000, 'min_calcium_mg': 1000, 'min_iron_mg': 17}
Structural Constraint: >= 3 items >= 500.0g
Parameters: Temp=1500.0, Cooling=0.997, Iterations=30000
Initial Diet: {6077: 231.5282112490936, 419: 199.91106048651017, 3452: 143.0554138328194, 6833: 216.94165864250726, 2049: 71.78601373569933, 6376: 127.73786601701909, 6288: 71.99465996736261, 3881: 102.52815256271086}
Initial Cost: 181075739.11

Finished. Final Best Cost: 150000000.00

--- Optimal Diet Found ---
Final Cost (Lower is Better): 150000000.00
Total Grams: 1596.4g

Foods (12 items):
- 289.1g of MILK,CND,EVAP,NONFAT,W/ ADDED VIT A & VITAMIN D
- 184.3g of BREADFRUIT SEEDS,ROASTED
- 180.0g of SEA LION,STELLER,HEART (ALASKA NATIVE)
- 169.3g of SEMOLINA,UNENRICHED
- 161.8g of SOUP,TOMATO BISQUE,CND,PREP W/ EQ VOLUME MILK
- 150.2g of CHICKEN